# Tokenization

- 문자열이란?
- String: Immutable Unicode codepoints

Unicode의 인코딩 방식대로 특정 바이트로 이루어진 배열

In [2]:
ord('ㄱ')

12593

# 1. Unicode로 인코딩하는 경우(Byte 단위)

In [3]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode('utf-8')
tokens = list(map(int, tokens))

print('======')
print(text)
print('length:', len(text))
print('======')
print(tokens)
print('length:', len(tokens))


Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 1

여러 Byte로 이루어진 문자의 경우 찢어짐 -> 길이 증가

자주 등장하는 연속된 바이트 쌍의 경우 하나의 토큰으로 여기는게 낫지 않을까?

--> common pair merging

# 2. Merging bytes

### 1. Simple Byte-Pair Encoding

In [4]:
# 연속된 두 바이트 쌍이 몇 번 등장하는지 확인
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    
    return counts

stats = get_stats(tokens)
print(stats)

# 가장 자주 등장한 쌍부터 내림차순 출력
print(sorted(((v, k) for k, v in stats.items()), reverse=True))

{(239, 188): 1, (188, 181): 1, (181, 239): 1, (239, 189): 6, (189, 142): 1, (142, 239): 1, (189, 137): 1, (137, 239): 1, (189, 131): 1, (131, 239): 1, (189, 143): 1, (143, 239): 1, (189, 132): 1, (132, 239): 1, (189, 133): 1, (133, 33): 1, (33, 32): 2, (32, 240): 3, (240, 159): 15, (159, 133): 7, (133, 164): 1, (164, 240): 1, (133, 157): 1, (157, 240): 1, (133, 152): 1, (152, 240): 1, (133, 146): 1, (146, 240): 1, (133, 158): 1, (158, 240): 1, (133, 147): 1, (147, 240): 1, (133, 148): 1, (148, 226): 1, (226, 128): 12, (128, 189): 1, (189, 32): 1, (159, 135): 7, (135, 186): 1, (186, 226): 1, (128, 140): 6, (140, 240): 6, (135, 179): 1, (179, 226): 1, (135, 174): 1, (174, 226): 1, (135, 168): 1, (168, 226): 1, (135, 180): 1, (180, 226): 1, (135, 169): 1, (169, 226): 1, (135, 170): 1, (170, 33): 1, (159, 152): 1, (152, 132): 1, (132, 32): 1, (32, 84): 1, (84, 104): 1, (104, 101): 6, (101, 32): 20, (32, 118): 1, (118, 101): 3, (101, 114): 6, (114, 121): 2, (121, 32): 2, (32, 110): 2, (110,

In [5]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [6]:
def merge(ids, pair, idx): # ids에 pair가 존재하는 경우 pair의 두 원소를 합쳐 idx로 변경경
    newids = []
    i = 0
    while i < len(ids):
        
        # 1. 가장 마지막에 있는 원소의 경우 제외-> OOB Error
        # 2. pair와 매칭되는 경우
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2

        # 아닌 경우 그냥 추가 
        else:
            newids.append(ids[i])
            i += 1
    return newids

print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

[5, 6, 99, 9, 1]


In [7]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print(len(tokens2))

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 226, 128, 156

성공적으로 줄인 모습을 볼 수 있음.

이제 이 과정을 loop을 통해 적절한 횟수만큼 반복해주자!

In [8]:
# 적당히 큰 크기의 텍스트 가져오기
text = """Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.

A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.

I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.

Diversity and Inherent Complexity
The Unicode Codespace
Codespace Allocation
Scripts
Usage Frequency
Encodings
UTF-8
UTF-16
Combining Marks
Canonical Equivalence
Normalization Forms
Grapheme Clusters
And More…
Diversity and Inherent Complexity
As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.

When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”

However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.

Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.

Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.

Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!

The Unicode Codespace
Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.

The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.

Codespace Allocation
To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.

Map of the Unicode codespace (click to zoom)

White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.

Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.

(In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)

Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.

Scripts
Let’s zoom in on the first three planes, since that’s where the action is:

Map of scripts in Unicode planes 0–2 (click to zoom)

This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.

Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).

Usage Frequency
One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.

Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)

You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.

Encodings
We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?

The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.

Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.

Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.

UTF-8
In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.

UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:

UTF-8 (binary)	Code point (binary)	Range
0xxxxxxx	xxxxxxx	U+0000–U+007F
110xxxxx 10yyyyyy	xxxxxyyyyyy	U+0080–U+07FF
1110xxxx 10yyyyyy 10zzzzzz	xxxxyyyyyyzzzzzz	U+0800–U+FFFF
11110xxx 10yyyyyy 10zzzzzz 10wwwwww	xxxyyyyyyzzzzzzwwwwww	U+10000–U+10FFFF
A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.

Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.

However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.

UTF-16
The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.

Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:

UTF-16 (binary)	Code point (binary)	Range
xxxxxxxxxxxxxxxx	xxxxxxxxxxxxxxxx	U+0000–U+FFFF
110110xxxxxxxxxx 110111yyyyyyyyyy	xxxxxxxxxxyyyyyyyyyy + 0x10000	U+10000–U+10FFFF
A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.

Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.

Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)

By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)

Combining Marks
In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!

Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.

In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.

If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.

For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.

Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.

Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓

A few other places where dynamic character composition shows up in Unicode:

Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.

A Hebrew example, with niqqud:	אֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד
Normal writing (no niqqud):	את דלתי הזיז הניע, קטב לשכתי ישוד
Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “​ि” = “हि” (“h” + “i” = “hi”).

Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”).

Canonical Equivalence
In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.

Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.

For example, the Vietnamese letter “ệ” can be expressed in five different ways:

Fully precomposed: U+1EC7 “ệ”
Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂”
Partially precomposed: U+00EA “ê” + U+0323 “◌̣”
Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂”
Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣”
Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!

Normalization Forms
To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).

The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)

The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).

There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.

Grapheme Clusters
As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.

UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.

The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.

Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.

And More…
There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.

Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.

Further reading:

The Unicode Standard
UTF-8 Everywhere Manifesto
Dark corners of Unicode by Eevee
ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things
Python 3 Unicode Howto
Google Noto Fonts—set of fonts intended to cover all assigned code points"""

tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print(len(tokens))

24532


In [9]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    
    return counts

def merge(ids, pair, idx): # ids에 pair가 존재하는 경우 pair의 두 원소를 합쳐 idx로 변경경
    newids = []
    i = 0
    while i < len(ids):
        
        # 1. 가장 마지막에 있는 원소의 경우 제외-> OOB Error
        # 2. pair와 매칭되는 경우
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2

        # 아닌 경우 그냥 추가 
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [10]:
vocab_size = 276 # 하이퍼 파라미터
num_merges = vocab_size - 256
ids = list(tokens) # 혹시 모르니 원본 복사

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f'merging {pair} into a new token {idx}')
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (101, 114) into a new token 260
merging (99, 111) into a new token 261
merging (116, 32) into a new token 262
merging (226, 128) into a new token 263
merging (44, 32) into a new token 264
merging (97, 110) into a new token 265
merging (111, 114) into a new token 266
merging (100, 32) into a new token 267
merging (97, 114) into a new token 268
merging (101, 110) into a new token 269
merging (257, 103) into a new token 270
merging (261, 100) into a new token 271
merging (121, 32) into a new token 272
merging (97, 108) into a new token 273
merging (259, 256) into a new token 274
merging (111, 110) into a new token 275


In [11]:
print('tokens length: ', len(tokens))
print('ids length: ', len(ids))
print(f'compression ration: {len(tokens) / len(ids):.2f}X')

tokens length:  24532
ids length:  19419
compression ration: 1.26X


이제 이 병합된 vocab을 통해 Encoding과 Decoding을 수행해보자!

### 2. Decoding

In [12]:
# token id -> byte obj 매핑 딕셔너리
vocab = {idx: bytes([idx]) for idx in range(256)}

# merge 쌍에 대해 돌며 바이트 결합
# merge가 내림차순 정렬되어있으므로 병합 우선순위가 높은 것부터 진행됨됨
for (p0, p1), idx in merges.items(): # python 3.7이상부턴 순서대로 수행되지만 이전 버전은 순서 보장 안됨!
    vocab[idx] = vocab[p0] + vocab[p1] # 바이트 덧셈(concat 연산)


def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors='replace') # utf-8 규칙에 어긋나는 바이트의 경우 특정 기호로 대체
    return text

print(decode([128]))

�


이런 식으로 디코딩하는 경우 LM이 token prediction을 잘못하면 �가 나올 수도 있음

### 3. Encoding

In [13]:
def encode(text):
    
    tokens = list(text.encode('utf-8'))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    
    return tokens

print(encode("hello world!"))

[104, 101, 108, 108, 111, 32, 119, 266, 108, 100, 33]


In [14]:
# 1. 짧은 문장의 경우
print(decode(encode("hello world!")))

# 2. 한 문자 / 빈 문장의 경우
print(encode('h'))

# 3. 훈련시킨 문장의 경우
text2 = decode(encode(text))
print(text2 == text)

# 4. 다른 긴 문장의 경우
valtext = """Unicode, formally The Unicode Standard,[note 1] is a text encoding standard maintained by the Unicode Consortium designed to support the use of text in all of the world's writing systems that can be digitized. Version 16.0 of the standard[A] defines 154998 characters and 168 scripts[3] used in various ordinary, literary, academic, and technical contexts.

Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes 3790 emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters.

Unicode has largely supplanted the previous environment of a myriad of incompatible character sets, each used within different locales and on different computer architectures. Unicode is used to encode the vast majority of text on the Internet, including most web pages, and relevant Unicode support has become a common consideration in contemporary software development.

The Unicode character repertoire is synchronized with ISO/IEC 10646, each being code-for-code identical with one another. However, The Unicode Standard is more than just a repertoire within which characters are assigned. To aid developers and designers, the standard also provides charts and reference data, as well as annexes explaining concepts germane to various scripts, providing guidance for their implementation. Topics covered by these annexes include character normalization, character composition and decomposition, collation, and directionality.[5]

Unicode text is processed and stored as binary data using one of several encodings, which define how to translate the standard's abstracted codes for characters into sequences of bytes. The Unicode Standard itself defines three encodings: UTF-8, UTF-16, and UTF-32, though several others exist. Of these, UTF-8 is the most widely used by a large margin, in part due to its backwards-compatibility with ASCII.

Origin and development
Unicode was originally designed with the intent of transcending limitations present in all text encodings designed up to that point: each encoding was relied upon for use in its own context, but with no particular expectation of compatibility with any other. Indeed, any two encodings chosen were often totally unworkable when used together, with text encoded in one interpreted as garbage characters by the other. Most encodings had only been designed to facilitate interoperation between a handful of scripts—often primarily between a given script and Latin characters—not between a large number of scripts, and not with all of the scripts supported being treated in a consistent manner.

The philosophy that underpins Unicode seeks to encode the underlying characters—graphemes and grapheme-like units—rather than graphical distinctions considered mere variant glyphs thereof, that are instead best handled by the typeface, through the use of markup, or by some other means. In particularly complex cases, such as the treatment of orthographical variants in Han characters, there is considerable disagreement regarding which differences justify their own encodings, and which are only graphical variants of other characters.

At the most abstract level, Unicode assigns a unique number called a code point to each character. Many issues of visual representation—including size, shape, and style—are intended to be up to the discretion of the software actually rendering the text, such as a web browser or word processor. However, partially with the intent of encouraging rapid adoption, the simplicity of this original model has become somewhat more elaborate over time, and various pragmatic concessions have been made over the course of the standard's development.

The first 256 code points mirror the ISO/IEC 8859-1 standard, with the intent of trivializing the conversion of text already written in Western European scripts. To preserve the distinctions made by different legacy encodings, therefore allowing for conversion between them and Unicode without any loss of information, many characters nearly identical to others, in both appearance and intended function, were given distinct code points. For example, the Halfwidth and Fullwidth Forms block encompasses a full semantic duplicate of the Latin alphabet, because legacy CJK encodings contained both "fullwidth" (matching the width of CJK characters) and "halfwidth" (matching ordinary Latin script) characters.

The Unicode Bulldog Award is given to people deemed to be influential in Unicode's development, with recipients including Tatsuo Kobayashi, Thomas Milo, Roozbeh Pournader, Ken Lunde, and Michael Everson.[6]

History
The origins of Unicode can be traced back to the 1980s, to a group of individuals with connections to Xerox's Character Code Standard (XCCS).[7] In 1987, Xerox employee Joe Becker, along with Apple employees Lee Collins and Mark Davis, started investigating the practicalities of creating a universal character set.[8] With additional input from Peter Fenwick and Dave Opstad,[7] Becker published a draft proposal for an "international/multilingual text character encoding system in August 1988, tentatively called Unicode". He explained that "the name 'Unicode' is intended to suggest a unique, unified, universal encoding".[7]

In this document, entitled Unicode 88, Becker outlined a scheme using 16-bit characters:[7]

Unicode is intended to address the need for a workable, reliable world text encoding. Unicode could be roughly described as "wide-body ASCII" that has been stretched to 16 bits to encompass the characters of all the world's living languages. In a properly engineered design, 16 bits per character are more than sufficient for this purpose.

This design decision was made based on the assumption that only scripts and characters in "modern" use would require encoding:[7]

Unicode gives higher priority to ensuring utility for the future than to preserving past antiquities. Unicode aims in the first instance at the characters published in the modern text (e.g. in the union of all newspapers and magazines printed in the world in 1988), whose number is undoubtedly far below 214 = 16,384. Beyond those modern-use characters, all others may be defined to be obsolete or rare; these are better candidates for private-use registration than for congesting the public list of generally useful Unicode.

In early 1989, the Unicode working group expanded to include Ken Whistler and Mike Kernaghan of Metaphor, Karen Smith-Yoshimura and Joan Aliprand of Research Libraries Group, and Glenn Wright of Sun Microsystems. In 1990, Michel Suignard and Asmus Freytag of Microsoft and NeXT's Rick McGowan had also joined the group. By the end of 1990, most of the work of remapping existing standards had been completed, and a final review draft of Unicode was ready.

The Unicode Consortium was incorporated in California on 3 January 1991,[9] and the first volume of The Unicode Standard was published that October. The second volume, now adding Han ideographs, was published in June 1992.

In 1996, a surrogate character mechanism was implemented in Unicode 2.0, so that Unicode was no longer restricted to 16 bits. This increased the Unicode codespace to over a million code points, which allowed for the encoding of many historic scripts, such as Egyptian hieroglyphs, and thousands of rarely used or obsolete characters that had not been anticipated for inclusion in the standard. Among these characters are various rarely used CJK characters—many mainly being used in proper names, making them far more necessary for a universal encoding than the original Unicode architecture envisioned.[10]

Version 1.0 of Microsoft's TrueType specification, published in 1992, used the name "Apple Unicode" instead of "Unicode" for the Platform ID in the naming table.

Unicode Consortium
Main article: Unicode Consortium
The Unicode Consortium is a nonprofit organization that coordinates Unicode's development. Full members include most of the main computer software and hardware companies (and few others) with any interest in text-processing standards, including Adobe, Apple, Google, IBM, Meta (previously as Facebook), Microsoft, Netflix, and SAP.[11]

Over the years several countries or government agencies have been members of the Unicode Consortium.[11]

The Consortium has the ambitious goal of eventually replacing existing character encoding schemes with Unicode and its standard Unicode Transformation Format (UTF) schemes, as many of the existing schemes are limited in size and scope and are incompatible with multilingual environments.

Scripts covered
Main article: Script (Unicode)

Many modern applications can render a substantial subset of the many scripts in Unicode, as demonstrated by this screenshot from the OpenOffice.org application.
Unicode currently covers most major writing systems in use today.[12][13]

As of 2024, a total of 168 scripts[14] are included in the latest version of Unicode (covering alphabets, abugidas and syllabaries), although there are still scripts that are not yet encoded, particularly those mainly used in historical, liturgical, and academic contexts. Further additions of characters to the already encoded scripts, as well as symbols, in particular for mathematics and music (in the form of notes and rhythmic symbols), also occur.

The Unicode Roadmap Committee (Michael Everson, Rick McGowan, Ken Whistler, V.S. Umamaheswaran)[15] maintain the list of scripts that are candidates or potential candidates for encoding and their tentative code block assignments on the Unicode Roadmap[16] page of the Unicode Consortium website. For some scripts on the Roadmap, such as Jurchen and Khitan large script, encoding proposals have been made and they are working their way through the approval process. For other scripts, such as Numidian and Rongorongo, no proposal has yet been made, and they await agreement on character repertoire and other details from the user communities involved.

Some modern invented scripts which have not yet been included in Unicode (e.g., Tengwar) or which do not qualify for inclusion in Unicode due to lack of real-world use (e.g., Klingon) are listed in the ConScript Unicode Registry, along with unofficial but widely used Private Use Areas code assignments.

There is also a Medieval Unicode Font Initiative focused on special Latin medieval characters. Part of these proposals has been already included in Unicode.

Script Encoding Initiative
The Script Encoding Initiative,[17] a project run by Deborah Anderson at the University of California, Berkeley was founded in 2002 with the goal of funding proposals for scripts not yet encoded in the standard. The project has become a major source of proposed additions to the standard in recent years.[18]

Versions
The Unicode Consortium together with the ISO have developed a shared repertoire following the initial publication of The Unicode Standard: Unicode and the ISO's Universal Coded Character Set (UCS) use identical character names and code points. However, the Unicode versions do differ from their ISO equivalents in two significant ways.

While the UCS is a simple character map, Unicode specifies the rules, algorithms, and properties necessary to achieve interoperability between different platforms and languages. Thus, The Unicode Standard includes more information, covering in-depth topics such as bitwise encoding, collation, and rendering. It also provides a comprehensive catalog of character properties, including those needed for supporting bidirectional text, as well as visual charts and reference data sets to aid implementers. Previously, The Unicode Standard was sold as a print volume containing the complete core specification, standard annexes,[note 2] and code charts. However, version 5.0, published in 2006, was the last version printed this way. Starting with version 5.2, only the core specification, published as a print-on-demand paperback, may be purchased.[19] The full text, on the other hand, is published as a free PDF on the Unicode website.

A practical reason for this publication method highlights the second significant difference between the UCS and Unicode—the frequency with which updated versions are released and new characters added. The Unicode Standard has regularly released annual expanded versions, occasionally with more than one version released in a calendar year and with rare cases where the scheduled release had to be postponed. For instance, in April 2020, a month after version 13.0 was published, the Unicode Consortium announced they had changed the intended release date for version 14.0, pushing it back six months to September 2021 due to the COVID-19 pandemic.

Unicode 16.0, the latest version, was released on 10 September 2024. It added 5,185 characters and seven new scripts: Garay, Gurung Khema, Kirat Rai, Ol Onal, Sunuwar, Todhri, and Tulu-Tigalari.[20]

Thus far, the following versions of The Unicode Standard have been published. Update versions, which do not include any changes to character repertoire, are signified by the third number (e.g., "version 4.0.1") and are omitted in the table below.[21]

Unicode version history and notable changes to characters and scripts
Version	Date	Publication
(book, text)	UCS edition	Total	Details
Scripts	Characters[a]
1.0.0[22]	October 1991	ISBN 0-201-56788-1 (vol. 1)	—	24	7129	Initial scripts covered: Arabic, Armenian, Bengali, Bopomofo, Cyrillic, Devanagari, Georgian, Greek and Coptic, Gujarati, Gurmukhi, Hangul, Hebrew, Hiragana, Kannada, Katakana, Lao, Latin, Malayalam, Odia, Tamil, Telugu, Thai, and Tibetan
1.0.1[23]	June 1992	ISBN 0-201-60845-6 (vol. 2)	25	28327+21204
−6	The initial 20,902 CJK Unified Ideographs
1.1[24]	June 1993	—	ISO/IEC 10646-1:1993
[b]

24	34168+5963
−9	33 reclassified as control characters. 4,306 Hangul syllables, Tibetan removed
2.0[25]	July 1996	ISBN 0-201-48345-9	25	38885+11373
−6656	Original set of Hangul syllables removed, new set of 11,172 Hangul syllables added at new location, Tibetan added back in a new location and with a different character repertoire, Surrogate character mechanism defined, Plane 15 and Plane 16 Private Use Areas allocated
2.1[26]	May 1998	—	38887+2
U+20AC € EURO SIGN, U+FFFC ￼ OBJECT REPLACEMENT CHARACTER[26]
3.0[27]	September 1999	ISBN 0-201-61633-5	ISO/IEC 10646-1:2000	38	49194+10307
Cherokee, Geʽez, Khmer, Mongolian, Burmese, Ogham, runes, Sinhala, Syriac, Thaana, Canadian Aboriginal syllabics, and Yi Syllables, Braille patterns
3.1[28]	March 2001	—	ISO/IEC 10646-1:2000[c]
ISO/IEC 10646-2:2001	41	94140+44946
Deseret, Gothic and Old Italic, sets of symbols for Western and Byzantine music, 42,711 additional CJK Unified Ideographs
3.2[29]	March 2002	45	95156+1016
Philippine scripts (Buhid, Hanunoo, Tagalog, and Tagbanwa)
4.0[30]	April 2003	ISBN 0-321-18578-1	ISO/IEC 10646:2003
[d]

52	96382+1226
Cypriot syllabary, Limbu, Linear B, Osmanya, Shavian, Tai Le, and Ugaritic, Hexagram symbols
4.1[31]	March 2005	—	59	97655+1273
Buginese, Glagolitic, Kharosthi, New Tai Lue, Old Persian, Sylheti Nagri, and Tifinagh, Coptic disunified from Greek, ancient Greek numbers and musical symbols, first named character sequences were introduced.[32]
5.0[33]	July 2006	ISBN 0-321-48091-0	64	99024+1369
Balinese, cuneiform, N'Ko, ʼPhags-pa, Phoenician[34]
5.1[35]	April 2008	—	75	100648+1624
Carian, Cham, Kayah Li, Lepcha, Lycian, Lydian, Ol Chiki, Rejang, Saurashtra, Sundanese, and Vai, sets of symbols for the Phaistos Disc, Mahjong tiles, Domino tiles, additions to Burmese, Scribal abbreviations, U+1E9E ẞ LATIN CAPITAL LETTER SHARP S
5.2[36]	October 2009	ISBN 978-1-936213-00-9	90	107296+6648
Avestan, Bamum, Gardiner's sign list of Egyptian hieroglyphs, Imperial Aramaic, Inscriptional Pahlavi, Inscriptional Parthian, Javanese, Kaithi, Lisu, Meetei Mayek, Old South Arabian, Old Turkic, Samaritan, Tai Tham and Tai Viet, additional CJK Unified Ideographs, Jamo for Old Hangul, Vedic Sanskrit
6.0[37]	October 2010	ISBN 978-1-936213-01-6	ISO/IEC 10646:2010
[e]

93	109384+2088
Batak, Brahmi, Mandaic, playing card symbols, transport and map symbols, alchemical symbols, emoticons and emoji,[38] additional CJK Unified Ideographs
6.1[39]	January 2012	ISBN 978-1-936213-02-3	ISO/IEC 10646:2012
[f]

100	110116+732
Chakma, Meroitic cursive, Meroitic hieroglyphs, Miao, Sharada, Sora Sompeng, and Takri
6.2[40]	September 2012	ISBN 978-1-936213-07-8	110117+1
U+20BA ₺ TURKISH LIRA SIGN
6.3[41]	September 2013	ISBN 978-1-936213-08-5	110122+5
5 bidirectional formatting characters
7.0[42]	June 2014	ISBN 978-1-936213-09-2	123	112956+2834
Bassa Vah, Caucasian Albanian, Duployan, Elbasan, Grantha, Khojki, Khudawadi, Linear A, Mahajani, Manichaean, Mende Kikakui, Modi, Mro, Nabataean, Old North Arabian, Old Permic, Pahawh Hmong, Palmyrene, Pau Cin Hau, Psalter Pahlavi, Siddham, Tirhuta, Warang Citi, and dingbats
8.0[43]	June 2015	ISBN 978-1-936213-10-8	ISO/IEC 10646:2014
[g]

129	120672+7716
Ahom, Anatolian hieroglyphs, Hatran, Multani, Old Hungarian, SignWriting, additional CJK Unified Ideographs, lowercase letters for Cherokee, 5 emoji skin tone modifiers
9.0[46]	June 2016	ISBN 978-1-936213-13-9	135	128172+7500
Adlam, Bhaiksuki, Marchen, Newa, Osage, Tangut, 72 emoji[47]
10.0[48]	June 2017	ISBN 978-1-936213-16-0	ISO/IEC 10646:2017
[h]

139	136690+8518
Zanabazar Square, Soyombo, Masaram Gondi, Nüshu, hentaigana, 7,494 CJK Unified Ideographs, 56 emoji, U+20BF ₿ BITCOIN SIGN
11.0[49]	June 2018	ISBN 978-1-936213-19-1	146	137374+684
Dogra, Georgian Mtavruli capital letters, Gunjala Gondi, Hanifi Rohingya, Indic Siyaq Numbers, Makasar, Medefaidrin, Old Sogdian and Sogdian, Maya numerals, 5 CJK Unified Ideographs, symbols for xiangqi and star ratings, 145 emoji
12.0[50]	March 2019	ISBN 978-1-936213-22-1	150	137928+554
Elymaic, Nandinagari, Nyiakeng Puachue Hmong, Wancho, Miao script, hiragana and katakana small letters, Tamil historic fractions and symbols, Lao letters for Pali, Latin letters for Egyptological and Ugaritic transliteration, hieroglyph format controls, 61 emoji
12.1[51]	May 2019	ISBN 978-1-936213-25-2	137929+1
U+32FF ㋿ SQUARE ERA NAME REIWA
13.0[52]	March 2020	ISBN 978-1-936213-26-9	ISO/IEC 10646:2020
[53]

154	143859+5930
Chorasmian, Dhives Akuru, Khitan small script, Yezidi, 4,969 CJK ideographs, Arabic script additions used to write Hausa, Wolof, and other African languages, additions used to write Hindko and Punjabi in Pakistan, Bopomofo additions used for Cantonese, Creative Commons license symbols, graphic characters for compatibility with teletext and home computer systems, 55 emoji
14.0[54]	September 2021	ISBN 978-1-936213-29-0	159	144697+838
Toto, Cypro-Minoan, Vithkuqi, Old Uyghur, Tangsa, extended IPA, Arabic script additions for use in languages across Africa and in Iran, Pakistan, Malaysia, Indonesia, Java, and Bosnia, additions for honorifics and Quranic use, additions to support languages in North America, the Philippines, India, and Mongolia, U+20C0 ⃀ SOM SIGN, Znamenny musical notation, 37 emoji
15.0[55]	September 2022	ISBN 978-1-936213-32-0	161	149186+4489
Kawi and Mundari, 20 emoji, 4,192 CJK ideographs, control characters for Egyptian hieroglyphs
15.1[56]	September 2023	ISBN 978-1-936213-33-7	149813+627
Additional CJK ideographs
16.0[57]	September 2024	ISBN 978-1-936213-34-4		168	154998+5185
Garay, Gurung Khema, Kirat Rai, Ol Onal, Sunuwar, Todhri, Tulu-Tigalari
 The total number of graphic and format characters, excluding private-use characters, control characters, noncharacters, and surrogate code points).
 2.0 added Amendments 5, 6, and 72.1 added two characters from Amendment 18.
 3.2 added Amendment 1.
 4.1 added Amendment 15.0 added Amendment 2 as well as four characters from Amendment 35.1 added Amendment 45.2 added Amendments 5 and 6
 Plus the Indian rupee sign
 6.2 added the Turkish lira sign6.3 added five additional characters7.0 added Amendments 1 and 2 as well as the ruble sign
 Plus Amendment 1, as well as the Lari sign, nine CJK unified ideographs, and 41 emoji;[44]
9.0 added Amendment 2, as well as Adlam, Newa, Japanese TV symbols, and 74 emoji and symbols.[45]
 Plus 56 emoji, 285 hentaigana characters, and 3 Zanabazar Square characters11.0 added 46 Mtavruli Georgian capital letters, 5 CJK unified ideographs, and 66 emoji12.0 added 62 additional characters.
Projected versions
The Unicode Consortium normally releases a new version of The Unicode Standard once a year. Version 17.0, the next major version, is projected to include 4301 new unified CJK characters.[58][59]

Architecture and terminology
See also: Universal Character Set characters
Codespace and code points
The Unicode Standard defines a codespace:[60] a sequence of integers called code points[61] in the range from 0 to 1114111, notated according to the standard as U+0000–U+10FFFF.[62] The codespace is a systematic, architecture-independent representation of The Unicode Standard; actual text is processed as binary data via one of several Unicode encodings, such as UTF-8.

In this normative notation, the two-character prefix U+ always precedes a written code point,[63] and the code points themselves are written as hexadecimal numbers. At least four hexadecimal digits are always written, with leading zeros prepended as needed. For example, the code point U+00F7 ÷ DIVISION SIGN is padded with two leading zeros, but U+13254 𓉔 EGYPTIAN HIEROGLYPH O004 () is not padded.[64]

There are a total of 220 + (216 − 211) = 1112064 valid code points within the codespace. (This number arises from the limitations of the UTF-16 character encoding, which can encode the 216 code points in the range U+0000 through U+FFFF except for the 211 code points in the range U+D800 through U+DFFF, which are used as surrogate pairs to encode the 220 code points in the range U+10000 through U+10FFFF.)

Code planes and blocks
Main article: Plane (Unicode)
The Unicode codespace is divided into 17 planes, numbered 0 to 16. Plane 0 is the Basic Multilingual Plane (BMP), and contains the most commonly used characters. All code points in the BMP are accessed as a single code unit in UTF-16 encoding and can be encoded in one, two or three bytes in UTF-8. Code points in planes 1 through 16 (the supplementary planes) are accessed as surrogate pairs in UTF-16 and encoded in four bytes in UTF-8.

Within each plane, characters are allocated within named blocks of related characters. The size of a block is always a multiple of 16, and is often a multiple of 128, but is otherwise arbitrary. Characters required for a given script may be spread out over several different, potentially disjunct blocks within the codespace.

General Category property
Each code point is assigned a classification, listed as the code point's General Category property. Here, at the uppermost level code points are categorized as one of Letter, Mark, Number, Punctuation, Symbol, Separator, or Other. Under each category, each code point is then further subcategorized. In most cases, other properties must be used to adequately describe all the characteristics of any given code point.

General Category (Unicode Character Property)[a]vte
Value	Category Major, minor	Basic type[b]	Character assigned[b]	Count[c]
(as of 16.0)	Remarks
 					
L, Letter; LC, Cased Letter (Lu, Ll, and Lt only)[d]
Lu	Letter, uppercase	Graphic	Character	1,858	
Ll	Letter, lowercase	Graphic	Character	2,258	
Lt	Letter, titlecase	Graphic	Character	31	Ligatures or digraphs containing an uppercase followed by a lowercase part (e.g., ǅ, ǈ, ǋ, and ǲ)
Lm	Letter, modifier	Graphic	Character	404	A modifier letter
Lo	Letter, other	Graphic	Character	136,477	An ideograph or a letter in a unicase alphabet
M, Mark
Mn	Mark, nonspacing	Graphic	Character	2,020	
Mc	Mark, spacing combining	Graphic	Character	468	
Me	Mark, enclosing	Graphic	Character	13	
N, Number
Nd	Number, decimal digit	Graphic	Character	760	All these, and only these, have Numeric Type = De[e]
Nl	Number, letter	Graphic	Character	236	Numerals composed of letters or letterlike symbols (e.g., Roman numerals)
No	Number, other	Graphic	Character	915	E.g., vulgar fractions, superscript and subscript digits, vigesimal digits
P, Punctuation
Pc	Punctuation, connector	Graphic	Character	10	Includes spacing underscore characters such as "_", and other spacing tie characters. Unlike other punctuation characters, these may be classified as "word" characters by regular expression libraries.[f]
Pd	Punctuation, dash	Graphic	Character	27	Includes several hyphen characters
Ps	Punctuation, open	Graphic	Character	79	Opening bracket characters
Pe	Punctuation, close	Graphic	Character	77	Closing bracket characters
Pi	Punctuation, initial quote	Graphic	Character	12	Opening quotation mark. Does not include the ASCII "neutral" quotation mark. May behave like Ps or Pe depending on usage
Pf	Punctuation, final quote	Graphic	Character	10	Closing quotation mark. May behave like Ps or Pe depending on usage
Po	Punctuation, other	Graphic	Character	640	
S, Symbol
Sm	Symbol, math	Graphic	Character	950	Mathematical symbols (e.g., +, −, =, ×, ÷, √, ∊, ≠). Does not include parentheses and brackets, which are in categories Ps and Pe. Also does not include !, *, -, or /, which despite frequent use as mathematical operators, are primarily considered to be "punctuation".
Sc	Symbol, currency	Graphic	Character	63	Currency symbols
Sk	Symbol, modifier	Graphic	Character	125	
So	Symbol, other	Graphic	Character	7,376	
Z, Separator
Zs	Separator, space	Graphic	Character	17	Includes the space, but not TAB, CR, or LF, which are Cc
Zl	Separator, line	Format	Character	1	Only U+2028 LINE SEPARATOR (LSEP)
Zp	Separator, paragraph	Format	Character	1	Only U+2029 PARAGRAPH SEPARATOR (PSEP)
C, Other
Cc	Other, control	Control	Character	65 (will never change)[e]	No name,[g] <control>
Cf	Other, format	Format	Character	170	Includes the soft hyphen, joining control characters (ZWNJ and ZWJ), control characters to support bidirectional text, and language tag characters
Cs	Other, surrogate	Surrogate	Not (only used in UTF-16)	2,048 (will never change)[e]	No name,[g] <surrogate>
Co	Other, private use	Private-use	Character (but no interpretation specified)	137,468 total (will never change)[e] (6,400 in BMP, 131,068 in Planes 15–16)	No name,[g] <private-use>
Cn	Other, not assigned	Noncharacter	Not	66 (will not change unless the range of Unicode code points is expanded)[e]	No name,[g] <noncharacter>
Reserved	Not	819,467	No name,[g] <reserved>
 "Table 4-4: General Category". The Unicode Standard. Unicode Consortium. September 2024.
 "Table 2-3: Types of code points". The Unicode Standard. Unicode Consortium. September 2024.
 "DerivedGeneralCategory.txt". The Unicode Consortium. 2024-04-30.
 "5.7.1 General Category Values". UTR #44: Unicode Character Database. Unicode Consortium. 2024-08-27.
 Unicode Character Encoding Stability Policies: Property Value Stability Stability policy: Some gc groups will never change. gc=Nd corresponds with Numeric Type=De (decimal).
 "Annex C: Compatibility Properties (§ word)". Unicode Regular Expressions. Version 23. Unicode Consortium. 2022-02-08. Unicode Technical Standard #18.
 "Table 4-9: Construction of Code Point Labels". The Unicode Standard. Unicode Consortium. September 2024. A Code Point Label may be used to identify a nameless code point. E.g. <control-hhhh>, <control-0088>. The Name remains blank, which can prevent inadvertently replacing, in documentation, a Control Name with a true Control code. Unicode also uses <not a character> for <noncharacter>.
The 1024 points in the range U+D800–U+DBFF are known as high-surrogate code points, and code points in the range U+DC00–U+DFFF (1024 code points) are known as low-surrogate code points. A high-surrogate code point followed by a low-surrogate code point forms a surrogate pair in UTF-16 in order to represent code points greater than U+FFFF. In principle, these code points cannot otherwise be used, though in practice this rule is often ignored, especially when not using UTF-16.

A small set of code points are guaranteed never to be assigned to characters, although third-parties may make independent use of them at their discretion. There are 66 of these noncharacters: U+FDD0–U+FDEF and the last two code points in each of the 17 planes (e.g. U+FFFE, U+FFFF, U+1FFFE, U+1FFFF, ..., U+10FFFE, U+10FFFF). The set of noncharacters is stable, and no new noncharacters will ever be defined.[65] Like surrogates, the rule that these cannot be used is often ignored, although the operation of the byte order mark assumes that U+FFFE will never be the first code point in a text. The exclusion of surrogates and noncharacters leaves 1111998 code points available for use.

Private-use code points are considered to be assigned, but they intentionally have no interpretation specified by The Unicode Standard[66] such that any interchange of such code points requires an independent agreement between the sender and receiver as to their interpretation. There are three private-use areas in the Unicode codespace:

Private Use Area: U+E000–U+F8FF (6400 characters),
Supplementary Private Use Area-A: U+F0000–U+FFFFD (65534 characters),
Supplementary Private Use Area-B: U+100000–U+10FFFD (65534 characters).
Graphic characters are those defined by The Unicode Standard to have particular semantics, either having a visible glyph shape or representing a visible space. As of Unicode 16.0, there are 154826 graphic characters.

Format characters are characters that do not have a visible appearance but may have an effect on the appearance or behavior of neighboring characters. For example, U+200C ZERO WIDTH NON-JOINER and U+200D ZERO WIDTH JOINER may be used to change the default shaping behavior of adjacent characters (e.g. to inhibit ligatures or request ligature formation). There are 172 format characters in Unicode 16.0.

65 code points, the ranges U+0000–U+001F and U+007F–U+009F, are reserved as control codes, corresponding to the C0 and C1 control codes as defined in ISO/IEC 6429. U+0089 LINE TABULATION, U+008A LINE FEED, and U+000D CARRIAGE RETURN are widely used in texts using Unicode. In a phenomenon known as mojibake, the C1 code points are improperly decoded according to the Windows-1252 codepage, previously widely used in Western European contexts.

Together, graphic, format, control code, and private use characters are collectively referred to as assigned characters. Reserved code points are those code points that are valid and available for use, but have not yet been assigned. As of Unicode 15.1, there are 819467 reserved code points.

Abstract characters 
Further information: Universal Character Set characters § Characters, grapheme clusters and glyphs
The set of graphic and format characters defined by Unicode does not correspond directly to the repertoire of abstract characters representable under Unicode. Unicode encodes characters by associating an abstract character with a particular code point.[67] However, not all abstract characters are encoded as a single Unicode character, and some abstract characters may be represented in Unicode by a sequence of two or more characters. For example, a Latin small letter "i" with an ogonek, a dot above, and an acute accent, which is required in Lithuanian, is represented by the character sequence U+012F; U+0307; U+0301. Unicode maintains a list of uniquely named character sequences for abstract characters that are not directly encoded in Unicode.[68]

All assigned characters have a unique and immutable name by which they are identified. This immutability has been guaranteed since version 2.0 of The Unicode Standard by its Name Stability policy.[65] In cases where a name is seriously defective and misleading, or has a serious typographical error, a formal alias may be defined that applications are encouraged to use in place of the official character name. For example, U+A015 ꀕ YI SYLLABLE WU has the formal alias YI SYLLABLE ITERATION MARK, and U+FE18 ︘ PRESENTATION FORM FOR VERTICAL RIGHT WHITE LENTICULAR BRAKCET (sic) has the formal alias PRESENTATION FORM FOR VERTICAL RIGHT WHITE LENTICULAR BRACKET.[69]

Ready-made versus composite characters
Unicode includes a mechanism for modifying characters that greatly extends the supported repertoire of glyphs. This covers the use of combining diacritical marks that may be added after the base character by the user. Multiple combining diacritics may be simultaneously applied to the same character. Unicode also contains precomposed versions of most letter/diacritic combinations in normal use. These make the conversion to and from legacy encodings simpler, and allow applications to use Unicode as an internal text format without having to implement combining characters. For example, é can be represented in Unicode as U+0065 e LATIN SMALL LETTER E followed by U+0301 ◌́ COMBINING ACUTE ACCENT), and equivalently as the precomposed character U+00E9 é LATIN SMALL LETTER E WITH ACUTE. Thus, users often have multiple equivalent ways of encoding the same character. The mechanism of canonical equivalence within The Unicode Standard ensures the practical interchangeability of these equivalent encodings.

An example of this arises with the Korean alphabet Hangul: Unicode provides a mechanism for composing Hangul syllables from their individual Hangul Jamo subcomponents. However, it also provides 11172 combinations of precomposed syllables made from the most common jamo.

CJK characters presently only have codes for uncomposable radicals and precomposed forms. Most Han characters have either been intentionally composed from, or reconstructed as compositions of, simpler orthographic elements called radicals, so in principle Unicode could have enabled their composition as it did with Hangul. While this could have greatly reduced the number of required code points, as well as allowing the algorithmic synthesis of many arbitrary new characters, the complexities of character etymologies and the post-hoc nature of radical systems add immense complexity to the proposal. Indeed, attempts to design CJK encodings on the basis of composing radicals have been met with difficulties resulting from the reality that Chinese characters do not decompose as simply or as regularly as Hangul does.

The CJK Radicals Supplement block is assigned to the range U+2E80–U+2EFF, and the Kangxi radicals are assigned to U+2F00–U+2FDF. The Ideographic Description Sequences block covers the range U+2FF0–U+2FFB, but The Unicode Standard warns against using its characters as an alternate representation for characters encoded elsewhere:

This process is different from a formal encoding of an ideograph. There is no canonical description of unencoded ideographs; there is no semantic assigned to described ideographs; there is no equivalence defined for described ideographs. Conceptually, ideographic descriptions are more akin to the English phrase "an 'e' with an acute accent on it" than to the character sequence <U+0065, U+0301>.

Ligatures

The Devanāgarī ddhrya-ligature (द् + ध् + र् + य = द्ध्र्य) of JanaSanskritSans[70]

The Arabic lām-alif ligature (ل ‎+‎ ا ‎=‎ لا)
Many scripts, including Arabic and Devanāgarī, have special orthographic rules that require certain combinations of letterforms to be combined into special ligature forms. The rules governing ligature formation can be quite complex, requiring special script-shaping technologies such as ACE (Arabic Calligraphic Engine by DecoType in the 1980s and used to generate all the Arabic examples in the printed editions of The Unicode Standard), which became the proof of concept for OpenType (by Adobe and Microsoft), Graphite (by SIL International), or AAT (by Apple).

Instructions are also embedded in fonts to tell the operating system how to properly output different character sequences. A simple solution to the placement of combining marks or diacritics is assigning the marks a width of zero and placing the glyph itself to the left or right of the left sidebearing (depending on the direction of the script they are intended to be used with). A mark handled this way will appear over whatever character precedes it, but will not adjust its position relative to the width or height of the base glyph; it may be visually awkward and it may overlap some glyphs. Real stacking is impossible but can be approximated in limited cases (for example, Thai top-combining vowels and tone marks can just be at different heights to start with). Generally, this approach is only effective in monospaced fonts but may be used as a fallback rendering method when more complex methods fail.

Standardized subsets
Several subsets of Unicode are standardized: Microsoft Windows since Windows NT 4.0 supports WGL-4 with 657 characters, which is considered to support all contemporary European languages using the Latin, Greek, or Cyrillic script. Other standardized subsets of Unicode include the Multilingual European Subsets:[71] MES-1 (Latin scripts only; 335 characters), MES-2 (Latin, Greek, and Cyrillic; 1062 characters)[72] and MES-3A & MES-3B (two larger subsets, not shown here). MES-2 includes every character in MES-1 and WGL-4.

The standard DIN 91379[73] specifies a subset of Unicode letters, special characters, and sequences of letters and diacritic signs to allow the correct representation of names and to simplify data exchange in Europe. This standard supports all of the official languages of all European Union countries, as well as the German minority languages and the official languages of Iceland, Liechtenstein, Norway, and Switzerland. To allow the transliteration of names in other writing systems to the Latin script according to the relevant ISO standards, all necessary combinations of base letters and diacritic signs are provided.

WGL-4, MES-1 and MES-2
Row	Cells	Range(s)
00	20–7E	Basic Latin (00–7F)
A0–FF	Latin-1 Supplement (80–FF)
01	00–13, 14–15, 16–2B, 2C–2D, 2E–4D, 4E–4F, 50–7E, 7F	Latin Extended-A (00–7F)
8F, 92, B7, DE-EF, FA–FF	Latin Extended-B (80–FF ...)
02	18–1B, 1E–1F	Latin Extended-B (... 00–4F)
59, 7C, 92	IPA Extensions (50–AF)
BB–BD, C6, C7, C9, D6, D8–DB, DC, DD, DF, EE	Spacing Modifier Letters (B0–FF)
03	74–75, 7A, 7E, 84–8A, 8C, 8E–A1, A3–CE, D7, DA–E1	Greek (70–FF)
04	00–5F, 90–91, 92–C4, C7–C8, CB–CC, D0–EB, EE–F5, F8–F9	Cyrillic (00–FF)
1E	02–03, 0A–0B, 1E–1F, 40–41, 56–57, 60–61, 6A–6B, 80–85, 9B, F2–F3	Latin Extended Additional (00–FF)
1F	00–15, 18–1D, 20–45, 48–4D, 50–57, 59, 5B, 5D, 5F–7D, 80–B4, B6–C4, C6–D3, D6–DB, DD–EF, F2–F4, F6–FE	Greek Extended (00–FF)
20	13–14, 15, 17, 18–19, 1A–1B, 1C–1D, 1E, 20–22, 26, 30, 32–33, 39–3A, 3C, 3E, 44, 4A	General Punctuation (00–6F)
7F, 82	Superscripts and Subscripts (70–9F)
A3–A4, A7, AC, AF	Currency Symbols (A0–CF)
21	05, 13, 16, 22, 26, 2E	Letterlike Symbols (00–4F)
5B–5E	Number Forms (50–8F)
90–93, 94–95, A8	Arrows (90–FF)
22	00, 02, 03, 06, 08–09, 0F, 11–12, 15, 19–1A, 1E–1F, 27–28, 29, 2A, 2B, 48, 59, 60–61, 64–65, 82–83, 95, 97	Mathematical Operators (00–FF)
23	02, 0A, 20–21, 29–2A	Miscellaneous Technical (00–FF)
25	00, 02, 0C, 10, 14, 18, 1C, 24, 2C, 34, 3C, 50–6C	Box Drawing (00–7F)
80, 84, 88, 8C, 90–93	Block Elements (80–9F)
A0–A1, AA–AC, B2, BA, BC, C4, CA–CB, CF, D8–D9, E6	Geometric Shapes (A0–FF)
26	3A–3C, 40, 42, 60, 63, 65–66, 6A, 6B	Miscellaneous Symbols (00–FF)
F0	(01–02)	Private Use Area (00–FF ...)
FB	01–02	Alphabetic Presentation Forms (00–4F)
FF	FD	Specials
Rendering software that cannot process a Unicode character appropriately often displays it as an open rectangle, or as U+FFFD to indicate the position of the unrecognized character. Some systems have made attempts to provide more information about such characters. Apple's Last Resort font will display a substitute glyph indicating the Unicode range of the character, and the SIL International's Unicode fallback font will display a box showing the hexadecimal scalar value of the character.

Mapping and encodings
Several mechanisms have been specified for storing a series of code points as a series of bytes.

Unicode defines two mapping methods: the Unicode Transformation Format (UTF) encodings, and the Universal Coded Character Set (UCS) encodings. An encoding maps (possibly a subset of) the range of Unicode code points to sequences of values in some fixed-size range, termed code units. All UTF encodings map code points to a unique sequence of bytes.[74] The numbers in the names of the encodings indicate the number of bits per code unit (for UTF encodings) or the number of bytes per code unit (for UCS encodings and UTF-1). UTF-8 and UTF-16 are the most commonly used encodings. UCS-2 is an obsolete subset of UTF-16; UCS-4 and UTF-32 are functionally equivalent.

UTF encodings include:

UTF-8, which uses one to four 8-bit units per code point,[note 3] and has maximal compatibility with ASCII
UTF-16, which uses one 16-bit unit per code point below U+010000, and a surrogate pair of two 16-bit units per code point in the range U+010000 to U+10FFFF
UTF-32, which uses one 32-bit unit per code point
UTF-EBCDIC, not specified as part of The Unicode Standard, which uses one to five 8-bit units per code point, intended to maximize compatibility with EBCDIC
UTF-8 uses one to four 8-bit units (bytes) per code point and, being compact for Latin scripts and ASCII-compatible, provides the de facto standard encoding for the interchange of Unicode text. It is used by FreeBSD and most recent Linux distributions as a direct replacement for legacy encodings in general text handling.

The UCS-2 and UTF-16 encodings specify the Unicode byte order mark (BOM) for use at the beginnings of text files, which may be used for byte-order detection (or byte endianness detection). The BOM, encoded as U+FEFF ZERO WIDTH NO-BREAK SPACE, has the important property of unambiguity on byte reorder, regardless of the Unicode encoding used; U+FFFE (the result of byte-swapping U+FEFF) does not equate to a legal character, and U+FEFF in places other than the beginning of text conveys the zero-width non-break space.

The same character converted to UTF-8 becomes the byte sequence EF BB BF. The Unicode Standard allows the BOM "can serve as a signature for UTF-8 encoded text where the character set is unmarked".[75] Some software developers have adopted it for other encodings, including UTF-8, in an attempt to distinguish UTF-8 from local 8-bit code pages. However RFC 3629, the UTF-8 standard, recommends that byte order marks be forbidden in protocols using UTF-8, but discusses the cases where this may not be possible. In addition, the large restriction on possible patterns in UTF-8 (for instance there cannot be any lone bytes with the high bit set) means that it should be possible to distinguish UTF-8 from other character encodings without relying on the BOM.

In UTF-32 and UCS-4, one 32-bit code unit serves as a fairly direct representation of any character's code point (although the endianness, which varies across different platforms, affects how the code unit manifests as a byte sequence). In the other encodings, each code point may be represented by a variable number of code units. UTF-32 is widely used as an internal representation of text in programs (as opposed to stored or transmitted text), since every Unix operating system that uses the gcc compilers to generate software uses it as the standard "wide character" encoding. Some programming languages, such as Seed7, use UTF-32 as an internal representation for strings and characters. Recent versions of the Python programming language (beginning with 2.2) may also be configured to use UTF-32 as the representation for Unicode strings, effectively disseminating such encoding in high-level coded software.

Punycode, another encoding form, enables the encoding of Unicode strings into the limited character set supported by the ASCII-based Domain Name System (DNS). The encoding is used as part of IDNA, which is a system enabling the use of Internationalized Domain Names in all scripts that are supported by Unicode. Earlier and now historical proposals include UTF-5 and UTF-6.

GB18030 is another encoding form for Unicode, from the Standardization Administration of China. It is the official character set of the People's Republic of China (PRC). BOCU-1 and SCSU are Unicode compression schemes. The April Fools' Day RFC of 2005 specified two parody UTF encodings, UTF-9 and UTF-18.

Adoption
See also: UTF-8 § Implementations and adoption
Unicode, in the form of UTF-8, has been the most common encoding for the World Wide Web since 2008.[76] It has near-universal adoption, and much of the non-UTF-8 content is found in other Unicode encodings, e.g. UTF-16. As of 2024, UTF-8 accounts for on average 98.3% of all web pages (and 983 of the top 1,000 highest-ranked web pages).[77] Although many pages only use ASCII characters to display content, UTF-8 was designed with 8-bit ASCII as a subset and almost no websites now declare their encoding to only be ASCII instead of UTF-8.[78] Over a third of the languages tracked have 100% UTF-8 use.

All internet protocols maintained by Internet Engineering Task Force, e.g. FTP,[79] have required support for UTF-8 since the publication of RFC 2277 in 1998, which specified that all IETF protocols "MUST be able to use the UTF-8 charset".[80]

Operating systems
Unicode has become the dominant scheme for the internal processing and storage of text. Although a great deal of text is still stored in legacy encodings, Unicode is used almost exclusively for building new information processing systems. Early adopters tended to use UCS-2 (the fixed-length two-byte obsolete precursor to UTF-16) and later moved to UTF-16 (the variable-length current standard), as this was the least disruptive way to add support for non-BMP characters. The best known such system is Windows NT (and its descendants, 2000, XP, Vista, 7, 8, 10, and 11), which uses UTF-16 as the sole internal character encoding. The Java and .NET bytecode environments, macOS, and KDE also use it for internal representation. Partial support for Unicode can be installed on Windows 9x through the Microsoft Layer for Unicode.

UTF-8 (originally developed for Plan 9)[81] has become the main storage encoding on most Unix-like operating systems (though others are also used by some libraries) because it is a relatively easy replacement for traditional extended ASCII character sets. UTF-8 is also the most common Unicode encoding used in HTML documents on the World Wide Web.

Multilingual text-rendering engines which use Unicode include Uniscribe and DirectWrite for Microsoft Windows, ATSUI and Core Text for macOS, and Pango for GTK+ and the GNOME desktop.

Input methods
Main article: Unicode input
Because keyboard layouts cannot have simple key combinations for all characters, several operating systems provide alternative input methods that allow access to the entire repertoire.

ISO/IEC 14755,[82] which standardises methods for entering Unicode characters from their code points, specifies several methods. There is the Basic method, where a beginning sequence is followed by the hexadecimal representation of the code point and the ending sequence. There is also a screen-selection entry method specified, where the characters are listed in a table on a screen, such as with a character map program.

Online tools for finding the code point for a known character include Unicode Lookup[83] by Jonathan Hedley and Shapecatcher[84] by Benjamin Milde. In Unicode Lookup, one enters a search key (e.g. "fractions"), and a list of corresponding characters with their code points is returned. In Shapecatcher, based on Shape context, one draws the character in a box and a list of characters approximating the drawing, with their code points, is returned.

Email
Main article: Unicode and email
MIME defines two different mechanisms for encoding non-ASCII characters in email, depending on whether the characters are in email headers (such as the "Subject:"), or in the text body of the message; in both cases, the original character set is identified as well as a transfer encoding. For email transmission of Unicode, the UTF-8 character set and the Base64 or the Quoted-printable transfer encoding are recommended, depending on whether much of the message consists of ASCII characters. The details of the two different mechanisms are specified in the MIME standards and generally are hidden from users of email software.

The IETF has defined[85][86] a framework for internationalized email using UTF-8, and has updated[87][88][89][90] several protocols in accordance with that framework.

The adoption of Unicode in email has been very slow.[citation needed] Some East Asian text is still encoded in encodings such as ISO-2022, and some devices, such as mobile phones,[citation needed] still cannot correctly handle Unicode data. Support has been improving, however. Many major free mail providers such as Yahoo! Mail, Gmail, and Outlook.com support it.

Web
Main article: Unicode and HTML
All W3C recommendations have used Unicode as their document character set since HTML 4.0. Web browsers have supported Unicode, especially UTF-8, for many years. There used to be display problems resulting primarily from font related issues; e.g. v6 and older of Microsoft Internet Explorer did not render many code points unless explicitly told to use a font that contains them.[91]

Although syntax rules may affect the order in which characters are allowed to appear, XML (including XHTML) documents, by definition,[92] comprise characters from most of the Unicode code points, with the exception of:

FFFE or FFFF.
most of the C0 control codes,
the permanently unassigned code points D800–DFFF,
HTML characters manifest either directly as bytes according to the document's encoding, if the encoding supports them, or users may write them as numeric character references based on the character's Unicode code point. For example, the references &#916;, &#1049;, &#1511;, &#1605;, &#3671;, &#12354;, &#21494;, &#33865;, and &#47568; (or the same numeric values expressed in hexadecimal, with &#x as the prefix) should display on all browsers as Δ, Й, ק ,م, ๗, あ, 叶, 葉, and 말.

When specifying URIs, for example as URLs in HTTP requests, non-ASCII characters must be percent-encoded.

Fonts
Main article: Unicode font
Unicode is not in principle concerned with fonts per se, seeing them as implementation choices.[93] Any given character may have many allographs, from the more common bold, italic and base letterforms to complex decorative styles. A font is "Unicode compliant" if the glyphs in the font can be accessed using code points defined in The Unicode Standard.[94] The standard does not specify a minimum number of characters that must be included in the font; some fonts have quite a small repertoire.

Free and retail fonts based on Unicode are widely available, since TrueType and OpenType support Unicode (and Web Open Font Format (WOFF and WOFF2) is based on those). These font formats map Unicode code points to glyphs, but OpenType and TrueType font files are restricted to 65,535 glyphs. Collection files provide a "gap mode" mechanism for overcoming this limit in a single font file. (Each font within the collection still has the 65,535 limit, however.) A TrueType Collection file would typically have a file extension of ".ttc".

Thousands of fonts exist on the market, but fewer than a dozen fonts—sometimes described as "pan-Unicode" fonts—attempt to support the majority of Unicode's character repertoire. Instead, Unicode-based fonts typically focus on supporting only basic ASCII and particular scripts or sets of characters or symbols. Several reasons justify this approach: applications and documents rarely need to render characters from more than one or two writing systems; fonts tend to demand resources in computing environments; and operating systems and applications show increasing intelligence in regard to obtaining glyph information from separate font files as needed, i.e., font substitution. Furthermore, designing a consistent set of rendering instructions for tens of thousands of glyphs constitutes a monumental task; such a venture passes the point of diminishing returns for most typefaces.

Newlines
Unicode partially addresses the newline problem that occurs when trying to read a text file on different platforms. Unicode defines a large number of characters that conforming applications should recognize as line terminators.

In terms of the newline, Unicode introduced U+2028 LINE SEPARATOR and U+2029 PARAGRAPH SEPARATOR. This was an attempt to provide a Unicode solution to encoding paragraphs and lines semantically, potentially replacing all of the various platform solutions. In doing so, Unicode does provide a way around the historical platform-dependent solutions. Nonetheless, few if any Unicode solutions have adopted these Unicode line and paragraph separators as the sole canonical line ending characters. However, a common approach to solving this issue is through newline normalization. This is achieved with the Cocoa text system in macOS and also with W3C XML and HTML recommendations. In this approach, every possible newline character is converted internally to a common newline (which one does not really matter since it is an internal operation just for rendering). In other words, the text system can correctly treat the character as a newline, regardless of the input's actual encoding.

Issues
Character unification
Han unification
Main article: Han unification
The Ideographic Research Group (IRG) is tasked with advising the Consortium and ISO regarding Han unification, or Unihan, especially the further addition of CJK unified and compatibility ideographs to the repertoire. The IRG is composed of experts from each region that has historically used Chinese characters. However, despite the deliberation within the committee, Han unification has consistently been one of the most contested aspects of The Unicode Standard since the genesis of the project.[95]

Existing character set standards such as the Japanese JIS X 0208 (encoded by Shift JIS) defined unification criteria, meaning rules for determining when a variant Chinese character is to be considered a handwriting/font difference (and thus unified), versus a spelling difference (to be encoded separately). Unicode's character model for CJK characters was based on the unification criteria used by JIS X 0208, as well as those developed by the Association for a Common Chinese Code in China.[96]

Due to the standard's principle of encoding semantic instead of stylistic variants, Unicode has received criticism for not assigning code points to certain rare and archaic kanji variants, possibly complicating processing of ancient and uncommon Japanese names. Since it places particular emphasis on Chinese, Japanese and Korean sharing many characters in common, Han unification is also sometimes perceived as treating the three as the same thing.[97] Regional differences in the expected forms of characters, in terms of typographical conventions and curricula for handwriting, do not always fall along language boundaries: although Hong Kong and Taiwan both write Chinese languages using Traditional Chinese characters, the preferred forms of characters differ between Hong Kong and Taiwan in some cases.[98]

Less-frequently-used alternative encodings exist, often predating Unicode, with character models differing from this paradigm, aimed at preserving the various stylistic differences between regional and/or nonstandard character forms. One example is the TRON Code favored by some users for handling historical Japanese text, though not widely adopted among the Japanese public. Another is the CCCII encoding adopted by library systems in Hong Kong, Taiwan and the United States. These have their own drawbacks in general use, leading to the Big5 encoding (introduced in 1984, four years after CCCII) having become more common than CCCII outside of library systems.[99] Although work at Apple based on Research Libraries Group's CJK Thesaurus, which was used to maintain the EACC variant of CCCII, was one of the direct predecessors of Unicode's Unihan set, Unicode adopted the JIS-style unification model.[96]

The earliest version of Unicode had a repertoire of fewer than 21,000 Han characters, largely limited to those in relatively common modern usage. As of version 16.0, the standard now encodes more than 97,000 Han characters, and work is continuing to add thousands more—largely historical and dialectal variant characters used throughout the Sinosphere.

Modern typefaces provide a means to address some of the practical issues in depicting unified Han characters with various regional graphical representations. The 'locl' OpenType table allows a renderer to select a different glyph for each code point based on the text locale.[100] The Unicode variation sequences can also provide in-text annotations for a desired glyph selection; this requires registration of the specific variant in the Ideographic Variation Database.

Italic or cursive characters in Cyrillic

Various Cyrillic characters shown with upright, oblique, and italic alternate forms
If the appropriate glyphs for characters in the same script differ only in the italic, Unicode has generally unified them, as can be seen in the comparison among a set of seven characters' italic glyphs as typically appearing in Russian, traditional Bulgarian, Macedonian, and Serbian texts at right, meaning that the differences are displayed through smart font technology or manually changing fonts. The same OpenType 'locl' technique is used.[101]

Localised case pairs
For use in the Turkish alphabet and Azeri alphabet, Unicode includes a separate dotless lowercase I (ı) and a dotted uppercase I (İ). However, the usual ASCII letters are used for the lowercase dotted I and the uppercase dotless I, matching how they are handled in the earlier ISO 8859-9. As such, case-insensitive comparisons for those languages have to use different rules than case-insensitive comparisons for other languages using the Latin script.[102][103] This can have security implications if, for example, sanitization code or access control relies on case-insensitive comparison.[103]

By contrast, the Icelandic eth (ð), the barred D (đ) and the retroflex D (ɖ), which usually[note 4] look the same in uppercase (Đ), are given the opposite treatment, and encoded separately in both letter-cases (in contrast to the earlier ISO 6937, which unifies the uppercase forms). Although it allows for case-insensitive comparison without needing to know the language of the text, this approach also has issues, requiring security measures relating to homoglyph attacks.[104]

Diacritics on lowercase I

Localised forms of the letter í (I with acute accent)
Whether the lowercase letter I is expected to retain its tittle when a diacritic applies also depends on local conventions.

Security
Unicode has a large number of homoglyphs, many of which look very similar or identical to ASCII letters. Substitution of these can make an identifier or URL that looks correct, but directs to a different location than expected.[105] Additionally, homoglyphs can also be used for manipulating the output of natural language processing (NLP) systems.[106] Mitigation requires disallowing these characters, displaying them differently, or requiring that they resolve to the same identifier;[107] all of this is complicated due to the huge and constantly changing set of characters.[108][109]

A security advisory was released in 2021 by two researchers, one from the University of Cambridge and the other from the University of Edinburgh, in which they assert that the BiDi marks can be used to make large sections of code do something different from what they appear to do. The problem was named "Trojan Source".[110] In response, code editors started highlighting marks to indicate forced text-direction changes.[111]

The UTF-8 and UTF-16 encodings do not accept all possible sequences of code units. Implementations vary in what they do when reading an invalid sequence, which has led to security bugs.[112] [113]

Mapping to legacy character sets
Unicode was designed to provide code-point-by-code-point round-trip format conversion to and from any preexisting character encodings, so that text files in older character sets can be converted to Unicode and then back and get back the same file, without employing context-dependent interpretation. That has meant that inconsistent legacy architectures, such as combining diacritics and precomposed characters, both exist in Unicode, giving more than one method of representing some text. This is most pronounced in the three different encoding forms for Korean Hangul. Since version 3.0, any precomposed characters that can be represented by a combined sequence of already existing characters can no longer be added to the standard to preserve interoperability between software using different versions of Unicode.

Injective mappings must be provided between characters in existing legacy character sets and characters in Unicode to facilitate conversion to Unicode and allow interoperability with legacy software. Lack of consistency in various mappings between earlier Japanese encodings such as Shift-JIS or EUC-JP and Unicode led to round-trip format conversion mismatches, particularly the mapping of the character JIS X 0208 '～' (1-33, WAVE DASH), heavily used in legacy database data, to either U+FF5E ～ FULLWIDTH TILDE (in Microsoft Windows) or U+301C 〜 WAVE DASH (other vendors).[114]

Some Japanese computer programmers objected to Unicode because it requires them to separate the use of U+005C \ REVERSE SOLIDUS (backslash) and U+00A5 ¥ YEN SIGN, which was mapped to 0x5C in JIS X 0201, and a lot of legacy code exists with this usage.[115] (This encoding also replaces tilde '~' 0x7E with macron '¯', now 0xAF.) The separation of these characters exists in ISO 8859-1, from long before Unicode.

Indic scripts
Further information: Tamil All Character Encoding
Indic scripts such as Tamil and Devanagari are each allocated only 128 code points, matching the ISCII standard. The correct rendering of Unicode Indic text requires transforming the stored logical order characters into visual order and the forming of ligatures (also known as conjuncts) out of components. Some local scholars argued in favor of assignments of Unicode code points to these ligatures, going against the practice for other writing systems, though Unicode contains some Arabic and other ligatures for backward compatibility purposes only.[116][117][118] Encoding of any new ligatures in Unicode will not happen, in part, because the set of ligatures is font-dependent, and Unicode is an encoding independent of font variations. The same kind of issue arose for the Tibetan script in 2003 when the Standardization Administration of China proposed encoding 956 precomposed Tibetan syllables,[119] but these were rejected for encoding by the relevant ISO committee (ISO/IEC JTC 1/SC 2).[120]

Thai alphabet support has been criticized for its ordering of Thai characters. The vowels เ, แ, โ, ใ, ไ that are written to the left of the preceding consonant are in visual order instead of phonetic order, unlike the Unicode representations of other Indic scripts. This complication is due to Unicode inheriting the Thai Industrial Standard 620, which worked in the same way, and was the way in which Thai had always been written on keyboards. This ordering problem complicates the Unicode collation process slightly, requiring table lookups to reorder Thai characters for collation.[97] Even if Unicode had adopted encoding according to spoken order, it would still be problematic to collate words in dictionary order. E.g., the word แสดง [sa dɛːŋ] "perform" starts with a consonant cluster "สด" (with an inherent vowel for the consonant "ส"), the vowel แ-, in spoken order would come after the ด, but in a dictionary, the word is collated as it is written, with the vowel following the ส.

Combining characters
Main article: Combining character
See also: Unicode normalization § Normalization
Characters with diacritical marks can generally be represented either as a single precomposed character or as a decomposed sequence of a base letter plus one or more non-spacing marks. For example, ḗ (precomposed e with macron and acute above) and ḗ (e followed by the combining macron above and combining acute above) should be rendered identically, both appearing as an e with a macron (◌̄) and acute accent (◌́), but in practice, their appearance may vary depending upon what rendering engine and fonts are being used to display the characters. Similarly, underdots, as needed in the romanization of Indic languages, will often be placed incorrectly.[citation needed] Unicode characters that map to precomposed glyphs can be used in many cases, thus avoiding the problem, but where no precomposed character has been encoded, the problem can often be solved by using a specialist Unicode font such as Charis SIL that uses Graphite, OpenType ('gsub'), or AAT technologies for advanced rendering features.

Anomalies
Main article: Unicode alias names and abbreviations
The Unicode Standard has imposed rules intended to guarantee stability.[121] Depending on the strictness of a rule, a change can be prohibited or allowed. For example, a "name" given to a code point cannot and will not change. But a "script" property is more flexible, by Unicode's own rules. In version 2.0, Unicode changed many code point "names" from version 1. At the same moment, Unicode stated that, thenceforth, an assigned name to a code point would never change. This implies that when mistakes are published, these mistakes cannot be corrected, even if they are trivial (as happened in one instance with the spelling BRAKCET for BRACKET in a character name). In 2006 a list of anomalies in character names was first published, and, as of June 2021, there were 104 characters with identified issues,[122] for example:

U+034F ͏ COMBINING GRAPHEME JOINER: Does not join graphemes.[122]
U+2118 ℘ SCRIPT CAPITAL P: This is a small letter. The capital is U+1D4AB 𝒫 MATHEMATICAL SCRIPT CAPITAL P.[123]
U+A015 ꀕ YI SYLLABLE WU: This is not a Yi syllable, but a Yi iteration mark.
U+FE18 ︘ PRESENTATION FORM FOR VERTICAL RIGHT WHITE LENTICULAR BRAKCET: bracket is spelled incorrectly.[124] (Spelling errors are resolved by using Unicode alias names.)
While Unicode defines the script designator (name) to be "Phags_Pa", in that script's character names, a hyphen is added: U+A840 ꡀ PHAGS-PA LETTER KA.[125][126] This, however, is not an anomaly, but the rule: hyphens are replaced by underscores in script designators.[125]"""
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

hello world!
[104]
True
True


# 3. Forced Splits using regex patterns (GPT)

병합되면 안되는 게 병합되는 경우 존재!
- 대표적인 예가 단어 + punctuation
- regex를 써서 강제로 병합을 막자!

In [26]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
# 's:   "he's","she's", "how's" 등  
# 't:   "won't", "wouldn't", "ain't" 등
# 're:  "you're" 등
# 've:  "I've", "We've" 등
# 'm:   "I'm" 등
# 'll:  "I'll", "you'll" 등
# 'd:   "She'd", "I'd" 등

# 위처럼 위따옴표의 경우를 하드코딩했기 때문에 많은 문제가 있음(대문자 인식 불가, 다른 유니코드의 따옴표 인식 불가 등)
# 영어에 한해서만 잘 됨

# \p{L} : 문자(어떤 언어든 상관 X)
#  ?\p{L}+ : 연속해서 등장하는 모든 문자들
# \p{N} : 숫자
#  ?\p{N}+ : 연속해서 등장하는 모든 숫자들

#  ?[^\s\p{L}\p{N}]+ : 숫자도 문자도 아닌 연속해서 등장하는 것들(주로 물음표, 느낌표 등)
# \s+(?!\S) : 공백 덩어리를 "마지막 공백만 제외하고" 
# \s+ : 문장 마지막 잉여 공백


print(re.findall(gpt2pat, "Hello Nathan3412, how's life? You're not gonna believe this. I just got attacked by 17,123 bees."))

['Hello', ' Nathan', '3412', ' how', "'s", ' life', ' You', "'re", ' not', ' gonna', ' believe', ' this', ' I', ' just', ' got', ' attacked', ' by', ' 17', '123', ' bees']


In [28]:
example = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print('FizzBuzz')
    elif i % 3 == 0:
        print('Fizz')
    elif i % 5 == 0:
        print('Buzz')
    else 
        print(i)
"""

print(re.findall(gpt2pat, example))

['\n', '\n', 'for', ' i', ' in', ' range', '1', ' 101', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', '\n       ', ' print', 'FizzBuzz', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', '\n       ', ' print', 'Fizz', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', '\n       ', ' print', 'Buzz', '\n   ', ' else', ' \n       ', ' print', 'i', '\n']


# 4. tiktoken by GPT

In [44]:
import tiktoken

# gpt2
enc = tiktoken.get_encoding('gpt2')
print(enc.encode(example))
print(enc.decode(enc.encode(example)))

# gpt4
enc = tiktoken.get_encoding('cl100k_base')
print(enc.encode(example))
print(enc.decode(enc.encode(example)))

[198, 198, 1640, 1312, 287, 2837, 7, 16, 11, 8949, 2599, 198, 220, 220, 220, 611, 1312, 4064, 513, 6624, 657, 290, 1312, 4064, 642, 6624, 657, 25, 198, 220, 220, 220, 220, 220, 220, 220, 3601, 10786, 37, 6457, 48230, 11537, 198, 220, 220, 220, 1288, 361, 1312, 4064, 513, 6624, 657, 25, 198, 220, 220, 220, 220, 220, 220, 220, 3601, 10786, 37, 6457, 11537, 198, 220, 220, 220, 1288, 361, 1312, 4064, 642, 6624, 657, 25, 198, 220, 220, 220, 220, 220, 220, 220, 3601, 10786, 48230, 11537, 198, 220, 220, 220, 2073, 220, 198, 220, 220, 220, 220, 220, 220, 220, 3601, 7, 72, 8, 198]


for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print('FizzBuzz')
    elif i % 3 == 0:
        print('Fizz')
    elif i % 5 == 0:
        print('Buzz')
    else 
        print(i)

[271, 2000, 602, 304, 2134, 7, 16, 11, 220, 4645, 997, 262, 422, 602, 1034, 220, 18, 624, 220, 15, 323, 602, 1034, 220, 20, 624, 220, 15, 512, 286, 1194, 493, 99615, 60573, 1329, 262, 4508, 602, 1034, 220, 18, 624, 220, 1

In [36]:
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
# !wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

import os, json

with open('./encoder/vocab.bpe') as f:
    bpe_data = f.read()
with open('./encoder/encoder.json') as f:
    encoder = json.load(f)

bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
print(bpe_merges)

[('Ġ', 't'), ('Ġ', 'a'), ('h', 'e'), ('i', 'n'), ('r', 'e'), ('o', 'n'), ('Ġt', 'he'), ('e', 'r'), ('Ġ', 's'), ('a', 't'), ('Ġ', 'w'), ('Ġ', 'o'), ('e', 'n'), ('Ġ', 'c'), ('i', 't'), ('i', 's'), ('a', 'n'), ('o', 'r'), ('e', 's'), ('Ġ', 'b'), ('e', 'd'), ('Ġ', 'f'), ('in', 'g'), ('Ġ', 'p'), ('o', 'u'), ('Ġa', 'n'), ('a', 'l'), ('a', 'r'), ('Ġt', 'o'), ('Ġ', 'm'), ('Ġo', 'f'), ('Ġ', 'in'), ('Ġ', 'd'), ('Ġ', 'h'), ('Ġan', 'd'), ('i', 'c'), ('a', 's'), ('l', 'e'), ('Ġt', 'h'), ('i', 'on'), ('o', 'm'), ('l', 'l'), ('en', 't'), ('Ġ', 'n'), ('Ġ', 'l'), ('s', 't'), ('Ġ', 're'), ('v', 'e'), ('Ġ', 'e'), ('r', 'o'), ('l', 'y'), ('Ġb', 'e'), ('Ġ', 'g'), ('Ġ', 'T'), ('c', 't'), ('Ġ', 'S'), ('i', 'd'), ('o', 't'), ('Ġ', 'I'), ('u', 't'), ('e', 't'), ('Ġ', 'A'), ('Ġ', 'is'), ('Ġ', 'on'), ('i', 'm'), ('a', 'm'), ('o', 'w'), ('a', 'y'), ('a', 'd'), ('s', 'e'), ('Ġth', 'at'), ('Ġ', 'C'), ('i', 'g'), ('Ġf', 'or'), ('a', 'c'), ('Ġ', 'y'), ('v', 'er'), ('u', 'r'), ('Ġ', 'u'), ('l', 'd'), ('Ġs', 't'), ('Ġ'

In [37]:
len(encoder)

50257

In [38]:
encoder['<|endoftext|>']

50256

In [42]:
import tiktoken

enc = tiktoken.get_encoding('cl100k_base')

print(enc.encode('안녕하세요'))
print(enc.decode(enc.encode('안녕하세요')))
print(enc.decode(enc.encode('안녕하세요')) == '안녕하세요')



[31495, 230, 75265, 243, 92245]
안녕하세요
True


# 5. SentencePiece

- 지금까지의 토큰화: utf 인코딩으로 byte로 바꾼 뒤 byte끼리 결합 및 인코딩
- sentencepiece: 인코딩 없이 unicode codepoint 레벨에서 결합 및 인코딩 

지금까지는 바이트 단위로 쪼갠 뒤 병합과 인코딩을 진행했지만, sentencepiece는 unicode 단계에서 병합과 인코딩을 진행함. 
그러므로 학습 때 보지 못한 문자가 들어오는 경우 그냥 바이트로 변환하던 기존의 방법과 달리, 어떻게 처리해야할지 모르게 되는 OOV 문제가 발생할 수 있음! (UNK 토큰의 필요성)

- character_coverage 옵션: 학습(vocab 구성) 단계에서 사용됨. 학습 텍스트 중 상위 빈도 몇%만큼만 사전에 추가(기본 0.9995)
- byte_fallback 옵션: 토큰화(사용) 단계에서 사용됨. 토큰화할 텍스트 중 OOV 토큰을 만났을 때 UNK로 처리할지, 바이트 단위로 쪼갠 뒤 처리할지 결정. 

주의: byte 단위로 쪼개도 해당 바이트가 사전에 없으면 그대로 UNK로 처리됨. 즉, UNK 토큰 개수만 쪼개져서 늘어날 뿐임

In [45]:
import sentencepiece as spm

with open('toy.txt', 'w', encoding='utf-8') as f:
    f.write('SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.')

In [ ]:
import os

options = dict(



    
)